In [9]:
df = pd.read_json('results.json')

ValueError: All arrays must be of the same length

In [65]:
import pandas as pd
import json
import os
# Read the JSON data as a dictionary
with open('results.json', 'r') as json_file:
    json_data = json.load(json_file)

# Convert the dictionary to a list of dictionaries
data_list = []
for class_name, items in json_data.items():
    for item in items:
        data_list.append({'Test_group': class_name, 'Test_name': item})
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data_list)
df


,Test_group,Test_name
0,TestReadWriteDiskValidator,testReadWriteDiskValidator
1,TestReadWriteDiskValidator,testCheckFailures
2,TestReadWriteDiskValidator,testReadWriteDiskValidator
3,TestReadWriteDiskValidator,testCheckFailures
4,TestReadWriteDiskValidator,testReadWriteDiskValidator
...,...,...
91,TestMRTimelineEventHandling,testMRNewTimelineServiceEventHandling
92,TestMRTimelineEventHandling,testMRNewTimelineServiceEventHandling
93,TestMRTimelineEventHandling,testMRNewTimelineServiceEventHandling
94,TestMRTimelineEventHandling,testMRNewTimelineServiceEventHandling


In [66]:
# Read the aborted-tests.txt file if it has some input in it
if os.path.getsize('aborted-tests.txt') > 1:
    df2 = pd.read_csv('aborted-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df2.columns = ['Test_group','Aborted_tests']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df2['Test_group'] = df2['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Merge df1 and df2
    df = pd.concat([df, df2] , ignore_index=True)
else:
    # Give the dataframe the column Aborted_tests with empty values
    df["Aborted_tests"] = None
    print("No aborted tests")

# Read the scala-tests.txt file if it has some input in it
if os.path.getsize('scala-tests.txt') > 1:
    df3 = pd.read_csv('scala-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df3.columns = ['Test_group','Test_name']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df3['Test_group'] = df3['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Merge new df1 and df3
    df = pd.merge(df, df3, how='outer')
else:
    # Give the dataframe the column Test_name with empty colmun
    df["Test_name"] = None
    print("No scala test errors")

# Concatinate the dataframe from the java tests with the new one
#df4 = pd.concat([df, df3], ignore_index=True)
df

,Test_group,Test_name,Aborted_tests
0,TestReadWriteDiskValidator,testReadWriteDiskValidator,NaN
1,TestReadWriteDiskValidator,testReadWriteDiskValidator,NaN
2,TestReadWriteDiskValidator,testReadWriteDiskValidator,NaN
3,TestReadWriteDiskValidator,testReadWriteDiskValidator,NaN
4,TestReadWriteDiskValidator,testCheckFailures,NaN
...,...,...,...
605,sql/hive,- Read/Write Hive ORC serde table *** FAILED ***,NaN
606,sql/hive,- SPARK-19459/SPARK-18220: read char/varchar c...,NaN
607,sql/hive,- SPARK-8020: set sql conf in spark conf *** F...,NaN
608,sql/hive,- SPARK-9757 Persist Parquet relation with dec...,NaN


In [67]:
import json
import pandas as pd
comparison_run = "0"
# Read the output from the transformer function
# Read the JSON data as a dictionary


# Load the external dataframe which we want to compare
# If We do not give a comparison run, we compare it with the same dataset which will not give any difference
if comparison_run == "0":
    df_external = df
# Otherwise we compare it with the dataset we give for comparison
else:
    df_external = pd.read_json(f'{comparison_run}')


# Drop the columns which can hinder the comparison
#df4 = df1.drop(columns=['Succeeded', 'Failed', 'Skipped', 'Pending'])
#df_external = df_external.drop(columns=['Succeeded', 'Failed', 'Skipped', 'Pending'])

# Merge ancient and new tables
df = df.merge(df_external, how='left', indicator=True)
df
# Checks if there is an aborted test or a test error in the new table

#df_comparison = df[((df['_merge'] == 'left_only') & pd.notna(df['Aborted_tests'])) | ((df['_merge'] == 'left_only') & pd.notna(df['Test_name']))]

#print("Comparison succeeded")

# Produce csv file for comparison and entire dataframe taht we will need later on for the decision making. These are the required outputs of the function.
#return df_comparison.to_csv('comparison.csv', header=False)

,Test_group,Test_name,Aborted_tests,_merge
0,TestReadWriteDiskValidator,testReadWriteDiskValidator,NaN,both
1,TestReadWriteDiskValidator,testReadWriteDiskValidator,NaN,both
2,TestReadWriteDiskValidator,testReadWriteDiskValidator,NaN,both
3,TestReadWriteDiskValidator,testReadWriteDiskValidator,NaN,both
4,TestReadWriteDiskValidator,testReadWriteDiskValidator,NaN,both
...,...,...,...,...
865,sql/hive,- Read/Write Hive ORC serde table *** FAILED ***,NaN,both
866,sql/hive,- SPARK-19459/SPARK-18220: read char/varchar c...,NaN,both
867,sql/hive,- SPARK-8020: set sql conf in spark conf *** F...,NaN,both
868,sql/hive,- SPARK-9757 Persist Parquet relation with dec...,NaN,both


In [63]:
df_comparison = df[((df['_merge'] == 'left_only') & pd.notna(df['Aborted_tests'])) | ((df['_merge'] == 'left_only') & pd.notna(df['Test_name']))]

In [56]:
df_comparison

,Test_group,Test_name,Aborted_tests,_merge


In [59]:
with open('results.json', 'r') as json_file:
    json_data = json.load(json_file)

# Convert the dictionary to a list of dictionaries
data_list = []
for class_name, items in json_data.items():
    for item in items:
        data_list.append({'Test_group': class_name, 'Test_name': item})
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data_list)

# Read the aborted-tests.txt file if it has some input in it
if os.path.getsize('aborted-tests.txt') > 1:
    df2 = pd.read_csv('aborted-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df2.columns = ['Test_group','Aborted_tests']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df2['Test_group'] = df2['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Concatinate df1 and df2
    df = pd.concat([df, df2] , ignore_index=True)
else:
    # Give the dataframe the column Aborted_tests with empty values
    df2["Aborted_tests"] = None
    print("No aborted tests")

# Read the scala-tests.txt file if it has some input in it
if os.path.getsize('scala-tests.txt') > 1:
    df3 = pd.read_csv('scala-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df3.columns = ['Test_group','Test_name']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df3['Test_group'] = df3['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Merge new df1 and df3
    df = pd.merge(df, df3, how='outer')
else:
    # Give the dataframe the column Test_name with empty colmun
    df["Test_name"] = None
    print("No scala test errors")
df

,Test_group,Test_name,Aborted_tests
0,TestReadWriteDiskValidator,testReadWriteDiskValidator,NaN
1,TestReadWriteDiskValidator,testReadWriteDiskValidator,NaN
2,TestReadWriteDiskValidator,testReadWriteDiskValidator,NaN
3,TestReadWriteDiskValidator,testReadWriteDiskValidator,NaN
4,TestReadWriteDiskValidator,testCheckFailures,NaN
...,...,...,...
605,sql/hive,- Read/Write Hive ORC serde table *** FAILED ***,NaN
606,sql/hive,- SPARK-19459/SPARK-18220: read char/varchar c...,NaN
607,sql/hive,- SPARK-8020: set sql conf in spark conf *** F...,NaN
608,sql/hive,- SPARK-9757 Persist Parquet relation with dec...,NaN


In [75]:
import pandas as pd
import sys
import os
def java_test_transfomer(build_number):
    """
    This function constructs the table with th test data taken from the extraction. It then returns the table in json format with all extracted information.
    """
    try:
        

        # Read the java-test-failures.txt file if it has some input in it
        if os.path.getsize('java-test-failures.txt') > 1:
            # Read output-tests.csv file as dataframe
            df2 = pd.read_csv('java-test-failures.txt', header = None, delimiter='/t', engine='python')
            # Name the single column Tests
            df2.columns = ['Tests']
            # Split the columns into two coulumns named Failed_test and Test_group
            df2[['Failed_test', 'Test_group']] = df2['Tests'].str.split('(', n=1, expand= True)
            # Remove the column Tests
            df2.drop(columns=['Tests'], inplace= True)
            # Remove all characters before .Test in coulumn Test_group
            df2['Test_group'] = df2['Test_group'].str.extract(r'(\.Test.*)', expand=False)
            # Only extract chracter between the . and °
            df2['Test_group'] = df2['Test_group'].str.extract(r'\.(.*?)\)', expand=False).str.strip()
            # Reorder columns
            df2= df2[['Test_group', 'Failed_test']]
            #df = pd.merge(df, df2, on = 'Test_group', how='outer')
        else:
            # Create the column Failed_test with empty values
            df2['Failed_test'] = None
            print("No java failed tests")

        
        # Write the rsultin json format where each element of Failed_test belongs to the class Test_group
        # create a dictionnary
        json_dict = {}
        # Go through each row
        for _, row in df2.iterrows():
            # define the columns
            class_name = row['Test_group']
            item_name = row['Failed_test']
            # if the classname does not yet exist in the dictionnary append it as a list
            if class_name not in json_dict:
                json_dict[class_name] = []
            # Append each item corresponding to the test belonging to the test class in the list of the test class
            json_dict[class_name].append(item_name)

        # Convert the dictionary to a JSON file
        json_data = pd.io.json.dumps(json_dict, indent=4)

        # Write the JSON data to a file
        with open('results.json', 'w') as json_file:
            json_file.write(json_data)
        
        print("Java data transformation succeeded")

    except:
        print("Java data transformation failed")
        sys.exit(1)

In [95]:
import json

build_number= 8

# Read the output from the scala transformer
#df = pd.read_json('results.json')

# Read the JSON data as a dictionary
with open('results.json', 'r') as json_file:
    json_data = json.load(json_file)

# Convert the dictionary to a list of dictionaries
data_list = []
for class_name, items in json_data.items():
    for item in items:
        data_list.append({'Test_group': class_name, 'Failed_test': item})
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data_list)

# Read the aborted-tests.txt file if it has some input in it
if os.path.getsize('aborted-tests.txt') > 1:
    df2 = pd.read_csv('aborted-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df2.columns = ['Test_group','Aborted_tests']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df2['Test_group'] = df2['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Merge df1 and df2
    df = pd.concat([df, df2] , ignore_index=True)
else:
    # Give the dataframe the column Aborted_tests with empty values
    df["Aborted_tests"] = None
    print("No aborted tests")

# Read the scala-tests.txt file if it has some input in it
if os.path.getsize('scala-tests.txt') > 1:
    df3 = pd.read_csv('scala-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df3.columns = ['Test_group','Failed_test']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df3['Test_group'] = df3['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Merge new df1 and df3
    df = pd.merge(df, df3, how='outer')
else:
    # Give the dataframe the column Failed_test with empty colmun
    df["Failed_test"] = None
    print("No scala test errors")
df

json_dict = {}
    # Go through each row
for _, row in df.iterrows():
    # define the columns
    class_name = row['Test_group']
    item_name = row['Failed_test']
    property_value = row['Aborted_tests']
    # if the classname does not yet exist in the dictionnary append it as a list
    if class_name not in json_dict:
        json_dict[class_name] = []
    # Append each item corresponding to the test belonging to the test class in the list of the test class
    json_dict[class_name].append({'Failed_test': item_name, 'Aborted_tests': property_value})

# Convert the dictionary to a JSON file
json_data = pd.io.json.dumps(json_dict, indent=4)

# Write the JSON data to a file
with open(f'results-{build_number}.json', 'w') as json_file:
    json_file.write(json_data)

# Concatinate the dataframe from the java tests with the new one
#df4 = pd.concat([df, df1], ignore_index=True)

# Write the rsultin json format where each element of Failed_test belongs to the class Test_group
# create a dictionnary


# The output of this function is the total results file combined with one from the java test function transformer
#return df4.to_json(f'results-{build_number}.json')


        

In [96]:
java_test_transfomer(8)
scala_transfomer_fy(8)
build_number= 5
comparison_run= 'results-5.json'
# Read the output from the transformer function
# Read the JSON data as a dictionary
with open(f'results-{build_number}.json', 'r') as json_file:
    json_data = json.load(json_file)

# Convert the dictionary to a list of dictionaries
data_list = []
for class_name, items in json_data.items():
    for item, item2 in items:
        data_list.append({'Test_group': class_name, 'Failed_test': item, 'Aborted_tests': item2})
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data_list)

# Load the external dataframe which we want to compare
# If We do not give a comparison run, we compare it with the same dataset which will not give any difference
if comparison_run == "0":
    df_external = df

df
#print(df_external)
#print(df)
# Merge ancient and new tables
#df = df.merge(df_external, how='left', indicator=True)
# Checks if there is an aborted test or a test error in the new table
#print(df[(df['_merge'] == 'left_only')])
#df_comparison = df[((df['_merge'] == 'left_only') & pd.notna(df['Aborted_tests'])) | ((df['_merge'] == 'left_only') & pd.notna(df['Failed_test']))]
#print("Comparison succeeded")
#print(df_comparison)
# Produce csv file for comparison and entire dataframe taht we will need later on for the decision making. These are the required outputs of the function.
#return df_comparison.to_csv('comparison.csv', header=False)
    

Java data transformation succeeded
Scala data transfromation succeeded


,Test_group,Failed_test,Aborted_tests
0,TestReadWriteDiskValidator,Failed_test,Aborted_tests
1,TestReadWriteDiskValidator,Failed_test,Aborted_tests
2,TestReadWriteDiskValidator,Failed_test,Aborted_tests
3,TestReadWriteDiskValidator,Failed_test,Aborted_tests
4,TestReadWriteDiskValidator,Failed_test,Aborted_tests
...,...,...,...
602,sql/hive,Failed_test,Aborted_tests
603,sql/hive,Failed_test,Aborted_tests
604,sql/hive,Failed_test,Aborted_tests
605,sql/hive,Failed_test,Aborted_tests


In [83]:
java_test_transfomer(8)
scala_transfomer_fy(8)
comparison_producer(8, 'results-5.json')

Java data transformation succeeded
Scala data transfromation succeeded
Comparison succeeded


In [53]:
import pandas as pd

df = pd.read_csv('output-tests.csv')
# Give column names
df.columns = ['Tests_run', 'Failures', 'Errors', 'Skipped', 'Test_group']
# Erase all unnecessary characters in each column and transform them into type int64
df['Tests_run'] = df['Tests_run'].str.split(':', n=1).str[1].astype('int64')
df['Failures'] = df['Failures'].str.split(':', n=1).str[1].astype('int64')
df['Errors'] = df['Errors'].str.split(':', n=1).str[1].astype('int64')
df['Skipped'] = df['Skipped'].str.split(':', n=1).str[1].astype('int64')
df['Test_group'] = df['Test_group'].str.split('-', n=1).str[1].str.replace('in','')
# delete lines where no testname is given which corresponds to the total columns
df = df.dropna(subset=['Test_group'])
# Add empty columns below to be compatible with schema
df['Pending'] = [None] * len(df)
df['Aborted_tests'] = [None] * len(df)
df['Failed']= df['Failures'] + df['Errors']
# Succeeded is the nuber of test runs minus failed and skipped test
df['Succeeded'] = df['Tests_run'] - df['Failed'] - df['Skipped']
# Select and reorder the columns
df= df[['Test_group', 'Succeeded', 'Failed', 'Skipped', 'Pending', 'Aborted_tests']]
df.to_csv('modified-output-tests.csv')


In [54]:
# Read output-tests.csv file as dataframe
df2 = pd.read_csv('java-test-failures.txt', header = None, delimiter='/t', engine='python')
# Name the single column Tests
df2.columns = ['Tests']
# Split the columns into two coulumns named Failed_test and Test_group
df2[['Failed_test', 'Test_group']] = df2['Tests'].str.split('(', n=1, expand= True)
# Remove the column Tests
df2.drop(columns=['Tests'], inplace= True)
# Split the columns Test_group into two after the )
df2[['Test_group', 'unnecsessary']] = df2['Test_group'].str.split(')', n=1, expand= True)
# Remove evrything after the parnetheses
df2.drop(columns=['unnecsessary'], inplace= True)
# Remove all characters before .Test in coulumn Test_group
#df2['Test_group'] = df2['Test_group'].str.extract('.*\((.*\d{4})\).*', expand=False)
# Only extract chracter between the . and °
#df2['Test_group'] = df2['Test_group'].str.extract('\.(.*?)\)', expand=False).str.strip()

#df2['Test_group'] = df2['Test_group'].str.split
# Reorder columns
df2= df2[['Test_group', 'Failed_test']]
df = pd.merge(df, df2, on = 'Test_group', how='outer')
#df2.to_csv("modified-java-test-failures.txt")
df

,Test_group,Succeeded,Failed,Skipped,Pending,Aborted_tests,Failed_test
0,org.apache.spark.util.kvstore.InMemoryStoreS...,6.0,0.0,0.0,None,None,NaN
1,org.apache.spark.util.kvstore.ArrayWrappersS...,1.0,0.0,0.0,None,None,NaN
2,org.apache.spark.util.kvstore.LevelDBIterato...,38.0,0.0,0.0,None,None,NaN
3,org.apache.spark.util.kvstore.InMemoryIterat...,38.0,0.0,0.0,None,None,NaN
4,org.apache.spark.util.kvstore.LevelDBTypeInf...,10.0,0.0,0.0,None,None,NaN
...,...,...,...,...,...,...,...
249,org.apache.hadoop.mapred.TestMRTimelineEventHa...,NaN,NaN,NaN,NaN,NaN,testMRNewTimelineServiceEventHandling
250,org.apache.hadoop.mapred.TestMRTimelineEventHa...,NaN,NaN,NaN,NaN,NaN,testMRNewTimelineServiceEventHandling
251,org.apache.hadoop.mapred.TestMRTimelineEventHa...,NaN,NaN,NaN,NaN,NaN,testMRNewTimelineServiceEventHandling
252,org.apache.hadoop.mapred.TestMRTimelineEventHa...,NaN,NaN,NaN,NaN,NaN,testMRNewTimelineServiceEventHandling


In [62]:
# read output-tests.csv file as dataframe
df = pd.read_csv('output-tests.csv')
# Give column names
df.columns = ['Tests_run', 'Failures', 'Errors', 'Skipped', 'Test_group']
# Erase all unnecessary characters in each column and transform them into type int64
df['Tests_run'] = df['Tests_run'].str.split(':', n=1).str[1].astype('int64')
df['Failures'] = df['Failures'].str.split(':', n=1).str[1].astype('int64')
df['Errors'] = df['Errors'].str.split(':', n=1).str[1].astype('int64')
df['Skipped'] = df['Skipped'].str.split(':', n=1).str[1].astype('int64')
df['Test_group'] = df['Test_group'].str.split('-', n=1).str[1].str.replace('in','')
# delete lines where no testname is given which corresponds to the total columns
df = df.dropna(subset=['Test_group'])
# Add empty columns below to be compatible with schema
df['Pending'] = [None] * len(df)
df['Aborted_tests'] = [None] * len(df)
df['Failed']= df['Failures'] + df['Errors']
# Succeeded is the nuber of test runs minus failed and skipped test
df['Succeeded'] = df['Tests_run'] - df['Failed'] - df['Skipped']
# Select and reorder the columns
df= df[['Test_group', 'Succeeded', 'Failed', 'Skipped', 'Pending', 'Aborted_tests']]

# Read the java-test-failures.txt file if it has some input in it
if os.path.getsize('java-test-failures.txt') > 1:
    # Read output-tests.csv file as dataframe
    df2 = pd.read_csv('java-test-failures.txt', header = None, delimiter='/t', engine='python')
    # Name the single column Tests
    df2.columns = ['Tests']
    # Split the columns into two coulumns named Failed_test and Test_group
    df2[['Failed_test', 'Test_group']] = df2['Tests'].str.split('(', n=1, expand= True)
    # Remove the column Tests
    df2.drop(columns=['Tests'], inplace= True)
    # Split the columns Test_group into two after the )
    df2[['Test_group', 'unnecsessary']] = df2['Test_group'].str.split(')', n=1, expand= True)
    # Remove evrything after the parnetheses
    df2.drop(columns=['unnecsessary'], inplace= True)
    # Reorder columns
    df2= df2[['Test_group', 'Failed_test']]
    df = pd.merge(df, df2, on = 'Test_group', how='outer')
else:
    # Create the column Failed_test with empty values
    df['Failed_test'] = None
    print("No java failed tests")

df.to_json("testing.json", orient="records")
df

,Test_group,Succeeded,Failed,Skipped,Pending,Aborted_tests,Failed_test
0,org.apache.spark.util.kvstore.InMemoryStoreS...,6.0,0.0,0.0,None,None,NaN
1,org.apache.spark.util.kvstore.ArrayWrappersS...,1.0,0.0,0.0,None,None,NaN
2,org.apache.spark.util.kvstore.LevelDBIterato...,38.0,0.0,0.0,None,None,NaN
3,org.apache.spark.util.kvstore.InMemoryIterat...,38.0,0.0,0.0,None,None,NaN
4,org.apache.spark.util.kvstore.LevelDBTypeInf...,10.0,0.0,0.0,None,None,NaN
...,...,...,...,...,...,...,...
249,org.apache.hadoop.mapred.TestMRTimelineEventHa...,NaN,NaN,NaN,NaN,NaN,testMRNewTimelineServiceEventHandling
250,org.apache.hadoop.mapred.TestMRTimelineEventHa...,NaN,NaN,NaN,NaN,NaN,testMRNewTimelineServiceEventHandling
251,org.apache.hadoop.mapred.TestMRTimelineEventHa...,NaN,NaN,NaN,NaN,NaN,testMRNewTimelineServiceEventHandling
252,org.apache.hadoop.mapred.TestMRTimelineEventHa...,NaN,NaN,NaN,NaN,NaN,testMRNewTimelineServiceEventHandling


In [65]:
df = pd.read_json("testing.json", orient= "records")


# Read the scala-end-results.txt file if it has some input in it
if os.path.getsize('scala-end-results.txt') > 1:
    # Read the scala-end-results.txt file
    df1 = pd.read_csv('scala-end-results.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df1.columns = ['Test_group','Results']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df1['Test_group'] = df1['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Iliminate all strings in Results column
    df1['Results'] = df1['Results'].str.replace("Tests: succeeded","").str.replace("failed","").str.replace("canceled","").str.replace("ignored","").str.replace("pending","")
    # Split the Results column into sevral columns
    splitted_columns = df1['Results'].str.split(',', expand=True)
    # Give column names
    splitted_columns.columns = ['Succeeded', 'Failed', 'Canceled', 'Ignored', 'Pending']
    # Transform string to integers
    splitted_columns = splitted_columns.astype(int)
    # Concatinate new df
    df1 = pd.concat([df1, splitted_columns], axis = 1).drop(columns = ['Results'])
    # Join the columns Canceled and Ignored in one signle coulumn
    df1['Skipped'] = df1['Canceled'] + df1['Ignored']
    # Drop the columns which are not needed anymore
    df1 = df1.drop(columns = ['Canceled', 'Ignored'])
    # Reorder the columns
    df1 = df1[['Test_group', 'Succeeded', 'Failed', 'Skipped', 'Pending']]
else:
    # Exit since no test has run
    columns = ['Test_group', 'Succeeded', 'Failed', 'Skipped', 'Pending']
    df1 = pd.DataFrame(columns = columns)
    print("No module has run")

# Read the aborted-tests.txt file if it has some input in it
if os.path.getsize('aborted-tests.txt') > 1:
    df2 = pd.read_csv('aborted-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df2.columns = ['Test_group','Aborted_tests']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df2['Test_group'] = df2['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Merge df1 and df2
    df1 = pd.concat([df1, df2] , ignore_index=True)
else:
    # Give the dataframe the column Aborted_tests with empty values
    df1["Aborted_tests"] = None
    print("No aborted tests")

# Read the scala-tests.txt file if it has some input in it
if os.path.getsize('scala-tests.txt') > 1:
    df3 = pd.read_csv('scala-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df3.columns = ['Test_group','Failed_test']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df3['Test_group'] = df3['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Merge new df1 and df3
    df1 = pd.merge(df1, df3, how='outer')
else:
    # Give the dataframe the column Failed_test with empty colmun
    df1["Failed_test"] = None
    print("No scala test errors")

# Merge df and df1
df = pd.concat([df, df1] , ignore_index=True)

,Test_group,Succeeded,Failed,Skipped,Pending,Aborted_tests,Failed_test
0,org.apache.spark.util.kvstore.InMemoryStoreS...,6.0,0.0,0.0,NaN,NaN,None
1,org.apache.spark.util.kvstore.ArrayWrappersS...,1.0,0.0,0.0,NaN,NaN,None
2,org.apache.spark.util.kvstore.LevelDBIterato...,38.0,0.0,0.0,NaN,NaN,None
3,org.apache.spark.util.kvstore.InMemoryIterat...,38.0,0.0,0.0,NaN,NaN,None
4,org.apache.spark.util.kvstore.LevelDBTypeInf...,10.0,0.0,0.0,NaN,NaN,None
...,...,...,...,...,...,...,...
786,sql/hive,2344.0,510.0,595.0,0.0,NaN,- SPARK-19459/SPARK-18220: read char/varchar c...
787,sql/hive,2344.0,510.0,595.0,0.0,NaN,- SPARK-8020: set sql conf in spark conf *** F...
788,sql/hive,2344.0,510.0,595.0,0.0,NaN,- SPARK-9757 Persist Parquet relation with dec...
789,sql/hive,2344.0,510.0,595.0,0.0,NaN,- SPARK-16901: set javax.jdo.option.Connection...


In [79]:
import json

# read output-tests.csv file as dataframe
df = pd.read_csv('output-tests.csv')
# Give column names
df.columns = ['Tests_run', 'Failures', 'Errors', 'Skipped', 'Test_group']
# Erase all unnecessary characters in each column and transform them into type int64
df['Tests_run'] = df['Tests_run'].str.split(':', n=1).str[1].astype('int64')
df['Failures'] = df['Failures'].str.split(':', n=1).str[1].astype('int64')
df['Errors'] = df['Errors'].str.split(':', n=1).str[1].astype('int64')
df['Skipped'] = df['Skipped'].str.split(':', n=1).str[1].astype('int64')
df['Test_group'] = df['Test_group'].str.split('-', n=1).str[1].str.replace('in','')
# delete lines where no testname is given which corresponds to the total columns
df = df.dropna(subset=['Test_group'])
# Add empty columns below to be compatible with schema
df['Pending'] = [None] * len(df)
df['Aborted_tests'] = [None] * len(df)
df['Failed']= df['Failures'] + df['Errors']
# Succeeded is the nuber of test runs minus failed and skipped test
df['Succeeded'] = df['Tests_run'] - df['Failed'] - df['Skipped']
# Select and reorder the columns
df= df[['Test_group', 'Succeeded', 'Failed', 'Skipped', 'Pending', 'Aborted_tests']]

# Read the java-test-failures.txt file if it has some input in it
if os.path.getsize('java-test-failures.txt') > 1:
    # Read output-tests.csv file as dataframe
    df2 = pd.read_csv('java-test-failures.txt', header = None, delimiter='/t', engine='python')
    # Name the single column Tests
    df2.columns = ['Tests']
    # Split the columns into two coulumns named Failed_test and Test_group
    df2[['Failed_test', 'Test_group']] = df2['Tests'].str.split('(', n=1, expand= True)
    # Remove the column Tests
    df2.drop(columns=['Tests'], inplace= True)
    # Split the columns Test_group into two after the )
    df2[['Test_group', 'unnecsessary']] = df2['Test_group'].str.split(')', n=1, expand= True)
    # Remove evrything after the parnetheses
    df2.drop(columns=['unnecsessary'], inplace= True)
    # Reorder columns
    df2= df2[['Test_group', 'Failed_test']]
    df = pd.merge(df, df2, on = 'Test_group', how='outer')
else:
    # Create the column Failed_test with empty values
    df['Failed_test'] = None
    print("No java failed tests")


nested_dict = {}
for _, row in df.iterrows():
    test_group = row['Test_group']
    if test_group not in nested_dict:
        nested_dict[test_group] = {
            'attributes': {
                'Succeeded': row['Succeeded'],
                'Failed': row['Failed'],
                'Skipped': row['Skipped'],
                'Pending': row['Pending'],
                'Aborted_tests' : row['Aborted_tests']
            },
            'Failed_tests': []
        }
    nested_dict[test_group]['Failed_tests'].append(row['Failed_test'])

# Write the nested dictionary to a JSON file
with open('output.json', 'w') as json_file:
    json.dump(nested_dict, json_file, indent=2)




In [81]:
# Read the scala-end-results.txt file if it has some input in it
if os.path.getsize('scala-end-results.txt') > 1:
    # Read the scala-end-results.txt file
    df1 = pd.read_csv('scala-end-results.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df1.columns = ['Test_group','Results']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df1['Test_group'] = df1['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Iliminate all strings in Results column
    df1['Results'] = df1['Results'].str.replace("Tests: succeeded","").str.replace("failed","").str.replace("canceled","").str.replace("ignored","").str.replace("pending","")
    # Split the Results column into sevral columns
    splitted_columns = df1['Results'].str.split(',', expand=True)
    # Give column names
    splitted_columns.columns = ['Succeeded', 'Failed', 'Canceled', 'Ignored', 'Pending']
    # Transform string to integers
    splitted_columns = splitted_columns.astype(int)
    # Concatinate new df
    df1 = pd.concat([df1, splitted_columns], axis = 1).drop(columns = ['Results'])
    # Join the columns Canceled and Ignored in one signle coulumn
    df1['Skipped'] = df1['Canceled'] + df1['Ignored']
    # Drop the columns which are not needed anymore
    df1 = df1.drop(columns = ['Canceled', 'Ignored'])
    # Reorder the columns
    df1 = df1[['Test_group', 'Succeeded', 'Failed', 'Skipped', 'Pending']]
else:
    # Exit since no test has run
    columns = ['Test_group', 'Succeeded', 'Failed', 'Skipped', 'Pending']
    df1 = pd.DataFrame(columns = columns)
    print("No module has run")

# Read the aborted-tests.txt file if it has some input in it
if os.path.getsize('aborted-tests.txt') > 1:
    df2 = pd.read_csv('aborted-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df2.columns = ['Test_group','Aborted_tests']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df2['Test_group'] = df2['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Merge df1 and df2
    df1 = pd.concat([df1, df2] , ignore_index=True)
else:
    # Give the dataframe the column Aborted_tests with empty values
    df1["Aborted_tests"] = None
    print("No aborted tests")

# Read the scala-tests.txt file if it has some input in it
if os.path.getsize('scala-tests.txt') > 1:
    df3 = pd.read_csv('scala-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df3.columns = ['Test_group','Failed_test']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df3['Test_group'] = df3['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Merge new df1 and df3
    df1 = pd.merge(df1, df3, how='outer')
else:
    # Give the dataframe the column Failed_test with empty colmun
    df1["Failed_test"] = None
    print("No scala test errors")

# Merge df and df1
#df = pd.concat([df, df1] , ignore_index=True)

# Read existing JSON data
with open('testing.json', 'r') as json_file:
    existing_data = json.load(json_file)


# Update existing data with new data
test_group = df1['Test_group']
if test_group not in existing_data:
    existing_data[test_group] = {
        'attributes': {
            'Succeeded': df1['Succeeded'],
            'Failed': df1['Failed'],
            'Skipped': df1['Skipped'],
            'Pending': df1['Pending'],
            'Aborted_tests': df1['Aborted_tests']
        },
        'Failed_tests': []
    }
existing_data[test_group]['Failed_tests'] += df1['Failed_test']

# Write the nested dictionary to a JSON file
with open(f'results-{build_number}.json', 'w') as json_file:
    json.dump(existing_data, json_file, indent=2)

TypeError: unhashable type: 'Series'

In [101]:
# Read the JSON file
with open('testing.json', 'r') as json_file:
    json_data = json.load(json_file)

# Convert the nested dictionary back into a DataFrame
records = []
for test_group, values in json_data.items():
    record = {'Test_group': test_group}
    record.update(values['attributes'])
    record['Failed_test'] = values['Failed_tests']
    records.append(record)

df = pd.DataFrame(records)
#df['Failed_test'] = df['Failed_test'].apply(lambda x: x[0])
df

,Test_group,Succeeded,Failed,Skipped,Pending,Aborted_tests,Failed_test
0,org.apache.spark.util.kvstore.InMemoryStoreS...,6.0,0.0,0.0,NaN,NaN,[nan]
1,org.apache.spark.util.kvstore.ArrayWrappersS...,1.0,0.0,0.0,NaN,NaN,[nan]
2,org.apache.spark.util.kvstore.LevelDBIterato...,38.0,0.0,0.0,NaN,NaN,[nan]
3,org.apache.spark.util.kvstore.InMemoryIterat...,38.0,0.0,0.0,NaN,NaN,[nan]
4,org.apache.spark.util.kvstore.LevelDBTypeInf...,10.0,0.0,0.0,NaN,NaN,[nan]
...,...,...,...,...,...,...,...
162,org.apache.hadoop.hdfs.TestFileChecksum,NaN,NaN,NaN,NaN,NaN,[testStripedFileChecksumWithMissedDataBlocksRa...
163,org.apache.hadoop.hdfs.TestFileChecksumComposi...,NaN,NaN,NaN,NaN,NaN,[testStripedFileChecksumWithMissedDataBlocksRa...
164,org.apache.hadoop.hdfs.TestReconstructStripedF...,NaN,NaN,NaN,NaN,NaN,"[testRecoverAnyBlocks1, testRecoverOneDataBloc..."
165,org.apache.hadoop.mapred.TestMRTimelineEventHa...,NaN,NaN,NaN,NaN,NaN,"[testMRNewTimelineServiceEventHandling, testMR..."


In [102]:
comparison_run = '0'
with open(f'results-{10}.json', 'r') as json_file:
            json_data = json.load(json_file)

# Convert the nested dictionary into a DataFrame
# Create a list called record
records = []
# Loop through each element of the json file
for test_group, values in json_data.items():
    # Get the class Test_group and define it as a variable test_group
    record = {'Test_group': test_group}
    # Get the attribute values
    record.update(values['attributes'])
    # Get the list of failed tests
    record['Failed_test'] = values['Failed_tests']
    # Append all these elements to the list records
    records.append(record)

# Define the dataframe
df = pd.DataFrame(records)

# Load the external dataframe which we want to compare
# If We do not give a comparison run, we compare it with the same dataset which will not give any difference
if comparison_run == "0":
    df_external = df
# Otherwise we compare it with the dataset we give for comparison
else:
    # Read the JSON data as a dictionary
    #with open(f'{comparison_run}', 'r') as json_file:
        #json_data = json.load(json_file)
    #df = pd.read_json(f'results-{comparison_run}.json', orient= "table")

    # Read the JSON data as a dictionary
    with open(f'{comparison_run}', 'r') as json_file:
        json_data = json.load(json_file)

    # Convert the nested dictionary into a DataFrame
    # Create a list called record
    records = []
    # Loop through each element of the json file
    for test_group, values in json_data.items():
        # Get the class Test_group and define it as a variable test_group
        record = {'Test_group': test_group}
        # Get the attribute values
        record.update(values['attributes'])
        # Get the list of failed tests
        record['Failed_test'] = values['Failed_tests']
        # Append all these elements to the list records
        records.append(record)

# Define the dataframe
df_external = pd.DataFrame(records)

df
# Merge ancient and new tables
#df = df.merge(df_external, how='left', indicator=True)
# Checks if there is an aborted test or a test error in the new table
#df[(df['_merge'] == 'left_only')]
#df_comparison = df[((df['_merge'] == 'left_only') & pd.notna(df['Aborted_tests'])) | ((df['_merge'] == 'left_only') & pd.notna(df['Failed_test']))]
#print("Comparison succeeded")
#print(df_comparison)

,Test_group,Succeeded,Failed,Skipped,Pending,Aborted_tests,Failed_test
0,org.apache.spark.util.kvstore.InMemoryStoreS...,6.0,0.0,0.0,NaN,NaN,[nan]
1,org.apache.spark.util.kvstore.ArrayWrappersS...,1.0,0.0,0.0,NaN,NaN,[nan]
2,org.apache.spark.util.kvstore.LevelDBIterato...,38.0,0.0,0.0,NaN,NaN,[nan]
3,org.apache.spark.util.kvstore.InMemoryIterat...,38.0,0.0,0.0,NaN,NaN,[nan]
4,org.apache.spark.util.kvstore.LevelDBTypeInf...,10.0,0.0,0.0,NaN,NaN,[nan]
...,...,...,...,...,...,...,...
189,external/kafka-0-10,15.0,0.0,0.0,0.0,NaN,[nan]
190,sql/catalyst,2611.0,1.0,2.0,0.0,NaN,[- daysToMillis and millisToDays *** FAILED ***]
191,sql/core,4109.0,1.0,59.0,0.0,NaN,[- SPARK-6330 regression test *** FAILED ***]
192,sql/hive,2344.0,510.0,595.0,0.0,NaN,[- SPARK-22745 - read Hive's statistics for pa...


In [104]:
with open('testing.json', 'r') as json_file:
    json_data = json.load(json_file)

# Create a list of dictionaries for each row
records = []
for group, data in json_data.items():
    attributes = data['attributes']
    for failed_test in data['Failed_tests']:
        record = {'Test_group': group}
        record.update(attributes)
        record['Failed_test'] = failed_test
        records.append(record)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(records)

df

,Test_group,Succeeded,Failed,Skipped,Pending,Aborted_tests,Failed_test
0,org.apache.spark.util.kvstore.InMemoryStoreS...,6.0,0.0,0.0,NaN,NaN,NaN
1,org.apache.spark.util.kvstore.ArrayWrappersS...,1.0,0.0,0.0,NaN,NaN,NaN
2,org.apache.spark.util.kvstore.LevelDBIterato...,38.0,0.0,0.0,NaN,NaN,NaN
3,org.apache.spark.util.kvstore.InMemoryIterat...,38.0,0.0,0.0,NaN,NaN,NaN
4,org.apache.spark.util.kvstore.LevelDBTypeInf...,10.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
249,org.apache.hadoop.mapred.TestMRTimelineEventHa...,NaN,NaN,NaN,NaN,NaN,testMRNewTimelineServiceEventHandling
250,org.apache.hadoop.mapred.TestMRTimelineEventHa...,NaN,NaN,NaN,NaN,NaN,testMRNewTimelineServiceEventHandling
251,org.apache.hadoop.mapred.TestMRTimelineEventHa...,NaN,NaN,NaN,NaN,NaN,testMRNewTimelineServiceEventHandling
252,org.apache.hadoop.mapred.TestMRTimelineEventHa...,NaN,NaN,NaN,NaN,NaN,testMRNewTimelineServiceEventHandling


In [105]:
# Read the output from the java transformer
with open('output.json', 'r') as json_file:
    json_data = json.load(json_file)

# Convert the nested dictionary into a DataFrame
# Create a list called record
    records = []
    # Loop through each element of the json file
    for test_group, values in json_data.items():
        # For each element in the failed_test list
        for failed_test in values['Failed_tests']:
            # Get the class Test_group and define it as a variable test_group
            record = {'Test_group': test_group}
            # Get the attribute values
            record.update(values['attributes'])
            # Get the list of failed tests
            record['Failed_test'] = failed_test
            # Append all these elements to the list records
            records.append(record)

# Define the dataframe
df = pd.DataFrame(records)



# Read the scala-end-results.txt file if it has some input in it
if os.path.getsize('scala-end-results.txt') > 1:
    # Read the scala-end-results.txt file
    df1 = pd.read_csv('scala-end-results.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df1.columns = ['Test_group','Results']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df1['Test_group'] = df1['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Iliminate all strings in Results column
    df1['Results'] = df1['Results'].str.replace("Tests: succeeded","").str.replace("failed","").str.replace("canceled","").str.replace("ignored","").str.replace("pending","")
    # Split the Results column into sevral columns
    splitted_columns = df1['Results'].str.split(',', expand=True)
    # Give column names
    splitted_columns.columns = ['Succeeded', 'Failed', 'Canceled', 'Ignored', 'Pending']
    # Transform string to integers
    splitted_columns = splitted_columns.astype(int)
    # Concatinate new df
    df1 = pd.concat([df1, splitted_columns], axis = 1).drop(columns = ['Results'])
    # Join the columns Canceled and Ignored in one signle coulumn
    df1['Skipped'] = df1['Canceled'] + df1['Ignored']
    # Drop the columns which are not needed anymore
    df1 = df1.drop(columns = ['Canceled', 'Ignored'])
    # Reorder the columns
    df1 = df1[['Test_group', 'Succeeded', 'Failed', 'Skipped', 'Pending']]
else:
    # Exit since no test has run
    columns = ['Test_group', 'Succeeded', 'Failed', 'Skipped', 'Pending']
    df1 = pd.DataFrame(columns = columns)
    print("No module has run")

# Read the aborted-tests.txt file if it has some input in it
if os.path.getsize('aborted-tests.txt') > 1:
    df2 = pd.read_csv('aborted-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df2.columns = ['Test_group','Aborted_tests']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df2['Test_group'] = df2['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Merge df1 and df2
    df1 = pd.concat([df1, df2] , ignore_index=True)
else:
    # Give the dataframe the column Aborted_tests with empty values
    df1["Aborted_tests"] = None
    print("No aborted tests")

# Read the scala-tests.txt file if it has some input in it
if os.path.getsize('scala-tests.txt') > 1:
    df3 = pd.read_csv('scala-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df3.columns = ['Test_group','Failed_test']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df3['Test_group'] = df3['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Merge new df1 and df3
    df1 = pd.merge(df1, df3, how='outer')
else:
    # Give the dataframe the column Failed_test with empty colmun
    df1["Failed_test"] = None
    print("No scala test errors")

# Concatinate dataframe from the java tests with df1
df = pd.concat([df, df1] , ignore_index=True)

# Create a dictionnary
nested_dict = {}
# Go through each row
for _, row in df.iterrows():
    # Define the Test_group which will be a class
    test_group = row['Test_group']
    # If the Test_group is not already in the dictionnary add it to it with its attributes and the list of failed tests
    if test_group not in nested_dict:
        nested_dict[test_group] = {
            'attributes': {
                'Succeeded': row['Succeeded'],
                'Failed': row['Failed'],
                'Skipped': row['Skipped'],
                'Pending': row['Pending'],
                'Aborted_tests' : row['Aborted_tests']
            },
            'Failed_tests': []
        }
    # Append individual failed tests if 'Failed_tests' is a list
    if isinstance(row['Failed_test'], list):
        nested_dict[test_group]['Failed_tests'].extend(row['Failed_test'])
    else:
        nested_dict[test_group]['Failed_tests'].append(row['Failed_test'])

# Write the nested dictionary to a JSON file
with open(f'results-{build_number}.json', 'w') as json_file:
    json.dump(nested_dict, json_file, indent=2)

TypeError: object of type 'float' has no len()

In [108]:
with open('testing.json', 'r') as json_file:
    json_data = json.load(json_file)

# Extract and analyze the keys and value types
schema = {}
for group, data in json_data.items():
    schema[group] = {}
    for key, value in data.items():
        value_type = type(value).__name__
        schema[group][key] = value_type

print(schema)

{'  org.apache.spark.util.kvstore.InMemoryStoreSuite': {'attributes': 'dict', 'Failed_tests': 'list'}, '  org.apache.spark.util.kvstore.ArrayWrappersSuite': {'attributes': 'dict', 'Failed_tests': 'list'}, '  org.apache.spark.util.kvstore.LevelDBIteratorSuite': {'attributes': 'dict', 'Failed_tests': 'list'}, '  org.apache.spark.util.kvstore.InMemoryIteratorSuite': {'attributes': 'dict', 'Failed_tests': 'list'}, '  org.apache.spark.util.kvstore.LevelDBTypeInfoSuite': {'attributes': 'dict', 'Failed_tests': 'list'}, '  org.apache.spark.util.kvstore.LevelDBSuite': {'attributes': 'dict', 'Failed_tests': 'list'}, '  org.apache.spark.network.util.TransportFrameDecoderSuite': {'attributes': 'dict', 'Failed_tests': 'list'}, '  org.apache.spark.network.util.NettyMemoryMetricsSuite': {'attributes': 'dict', 'Failed_tests': 'list'}, '  org.apache.spark.network.util.CryptoUtilsSuite': {'attributes': 'dict', 'Failed_tests': 'list'}, '  org.apache.spark.network.TransportResponseHandlerSuite': {'attribu

In [109]:
pd.read_json('results-10.json')

,org.apache.spark.util.kvstore.InMemoryStoreSuite,org.apache.spark.util.kvstore.ArrayWrappersSuite,org.apache.spark.util.kvstore.LevelDBIteratorSuite,org.apache.spark.util.kvstore.InMemoryIteratorSuite,org.apache.spark.util.kvstore.LevelDBTypeInfoSuite,org.apache.spark.util.kvstore.LevelDBSuite,org.apache.spark.network.util.TransportFrameDecoderSuite,org.apache.spark.network.util.NettyMemoryMetricsSuite,org.apache.spark.network.util.CryptoUtilsSuite,org.apache.spark.network.TransportResponseHandlerSuite,...,external/flume-assembly,external/flume-sink,external/flume,external/kafka-0-10-assembly,external/kafka-0-10-sql,external/kafka-0-10,sql/catalyst,sql/core,sql/hive,sql/hive-thriftserver
attributes,"{'Succeeded': 6.0, 'Failed': 0.0, 'Skipped': 0...","{'Succeeded': 1.0, 'Failed': 0.0, 'Skipped': 0...","{'Succeeded': 38.0, 'Failed': 0.0, 'Skipped': ...","{'Succeeded': 38.0, 'Failed': 0.0, 'Skipped': ...","{'Succeeded': 10.0, 'Failed': 0.0, 'Skipped': ...","{'Succeeded': 8.0, 'Failed': 0.0, 'Skipped': 0...","{'Succeeded': 6.0, 'Failed': 0.0, 'Skipped': 0...","{'Succeeded': 2.0, 'Failed': 0.0, 'Skipped': 0...","{'Succeeded': 1.0, 'Failed': 0.0, 'Skipped': 0...","{'Succeeded': 8.0, 'Failed': 0.0, 'Skipped': 0...",...,"{'Succeeded': 0.0, 'Failed': 0.0, 'Skipped': 0...","{'Succeeded': 5.0, 'Failed': 0.0, 'Skipped': 0...","{'Succeeded': 4.0, 'Failed': 0.0, 'Skipped': 0...","{'Succeeded': 0.0, 'Failed': 0.0, 'Skipped': 0...","{'Succeeded': 105.0, 'Failed': 0.0, 'Skipped':...","{'Succeeded': 15.0, 'Failed': 0.0, 'Skipped': ...","{'Succeeded': 2611.0, 'Failed': 1.0, 'Skipped'...","{'Succeeded': 4109.0, 'Failed': 1.0, 'Skipped'...","{'Succeeded': 2344.0, 'Failed': 510.0, 'Skippe...","{'Succeeded': 40.0, 'Failed': 0.0, 'Skipped': ..."
Failed_tests,[None],[None],[None],[None],[None],[None],[None],[None],[None],[None],...,[None],[None],[None],[None],[None],[None],[- daysToMillis and millisToDays *** FAILED ***],[- SPARK-6330 regression test *** FAILED ***],[- SPARK-22745 - read Hive's statistics for pa...,[None]


In [140]:
# Read the scala-end-results.txt file if it has some input in it
if os.path.getsize('scala-end-results.txt') > 1:
    # Read the scala-end-results.txt file
    df1 = pd.read_csv('scala-end-results.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df1.columns = ['Test_group','Results']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df1['Test_group'] = df1['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Iliminate all strings in Results column
    df1['Results'] = df1['Results'].str.replace("Tests: succeeded","").str.replace("failed","").str.replace("canceled","").str.replace("ignored","").str.replace("pending","")
    # Split the Results column into sevral columns
    splitted_columns = df1['Results'].str.split(',', expand=True)
    # Give column names
    splitted_columns.columns = ['Succeeded', 'Failed', 'Canceled', 'Ignored', 'Pending']
    # Transform string to integers
    splitted_columns = splitted_columns.astype(int)
    # Concatinate new df
    df1 = pd.concat([df1, splitted_columns], axis = 1).drop(columns = ['Results'])
    # Join the columns Canceled and Ignored in one signle coulumn
    df1['Skipped'] = df1['Canceled'] + df1['Ignored']
    # Drop the columns which are not needed anymore
    df1 = df1.drop(columns = ['Canceled', 'Ignored'])
    # Reorder the columns
    df1 = df1[['Test_group', 'Succeeded', 'Failed', 'Skipped', 'Pending']]
else:
    # Exit since no test has run
    columns = ['Test_group', 'Succeeded', 'Failed', 'Skipped', 'Pending']
    df1 = pd.DataFrame(columns = columns)
    print("No module has run")

# Read the aborted-tests.txt file if it has some input in it
if os.path.getsize('aborted-tests.txt') > 1:
    df2 = pd.read_csv('aborted-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df2.columns = ['Test_group','Aborted_tests']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df2['Test_group'] = df2['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Merge df1 and df2
    df1 = pd.concat([df1, df2] , ignore_index=True)
else:
    # Give the dataframe the column Aborted_tests with empty values
    df1["Aborted_tests"] = None
    print("No aborted tests")

# Read the scala-tests.txt file if it has some input in it
if os.path.getsize('scala-tests.txt') > 1:
    df3 = pd.read_csv('scala-tests.txt', header=None, sep='.txt:', engine='python')
    # Give column names
    df3.columns = ['Test_group','Failed_test']
    # Iliminate the path to SparkTestSuite file and keep only the module name 
    df3['Test_group'] = df3['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
    # Merge new df1 and df3
    df1 = pd.merge(df1, df3, how='outer')
else:
    # Give the dataframe the column Failed_test with empty colmun
    df1["Failed_test"] = None
    print("No scala test errors")

df1[df1['Test_group']=='tools']

,Test_group,Succeeded,Failed,Skipped,Pending,Aborted_tests,Failed_test
535,tools,NaN,NaN,NaN,NaN,*** RUN ABORTED ***,NaN


In [130]:
# read output-tests.csv file as dataframe
df = pd.read_csv('output-tests.csv')
# Give column names
df.columns = ['Tests_run', 'Failures', 'Errors', 'Skipped', 'Test_group']
# Erase all unnecessary characters in each column and transform them into type int64
df['Tests_run'] = df['Tests_run'].str.split(':', n=1).str[1].astype('int64')
df['Failures'] = df['Failures'].str.split(':', n=1).str[1].astype('int64')
df['Errors'] = df['Errors'].str.split(':', n=1).str[1].astype('int64')
df['Skipped'] = df['Skipped'].str.split(':', n=1).str[1].astype('int64')
df['Test_group'] = df['Test_group'].str.split('-', n=1).str[1].str.replace('in','')
# delete lines where no testname is given which corresponds to the total columns
df = df.dropna(subset=['Test_group'])
# Add empty columns below to be compatible with schema
df['Pending'] = [None] * len(df)
df['Aborted_tests'] = [None] * len(df)
df['Failed']= df['Failures'] + df['Errors']
# Succeeded is the nuber of test runs minus failed and skipped test
df['Succeeded'] = df['Tests_run'] - df['Failed'] - df['Skipped']
# Select and reorder the columns
df= df[['Test_group', 'Succeeded', 'Failed', 'Skipped', 'Pending', 'Aborted_tests']]

# Read the java-test-failures.txt file if it has some input in it
if os.path.getsize('java-test-failures.txt') > 1:
    # Read output-tests.csv file as dataframe
    df2 = pd.read_csv('java-test-failures.txt', header = None, delimiter='/t', engine='python')
    # Name the single column Tests
    df2.columns = ['Tests']
    # Split the columns into two coulumns named Failed_test and Test_group
    df2[['Failed_test', 'Test_group']] = df2['Tests'].str.split('(', n=1, expand= True)
    # Remove the column Tests
    df2.drop(columns=['Tests'], inplace= True)
    # Split the columns Test_group into two after the )
    df2[['Test_group', 'unnecsessary']] = df2['Test_group'].str.split(')', n=1, expand= True)
    # Remove evrything after the parnetheses
    df2.drop(columns=['unnecsessary'], inplace= True)
    # Reorder columns
    df2= df2[['Test_group', 'Failed_test']]
    df = pd.merge(df, df2, on = 'Test_group', how='outer')
else:
    # Create the column Failed_test with empty values
    df['Failed_test'] = None
    print("No java failed tests")

df2 = df2.drop_duplicates()
df2 = df2.drop_duplicates()
df2

,Test_group,Failed_test
0,org.apache.hadoop.util.TestReadWriteDiskValidator,testReadWriteDiskValidator
1,org.apache.hadoop.util.TestReadWriteDiskValidator,testCheckFailures
8,org.apache.hadoop.io.compress.TestCompressorDe...,testCompressorDecompressor
9,org.apache.hadoop.io.compress.TestCompressorDe...,testCompressorDecompressorWithExeedBufferLimit
16,org.apache.hadoop.io.compress.snappy.TestSnapp...,testSnappyCompressDecompressInMultiThreads
17,org.apache.hadoop.io.compress.snappy.TestSnapp...,testSnappyCompressDecompress
24,org.apache.hadoop.security.TestLdapGroupsMapping,testLdapReadTimeout
25,org.apache.hadoop.security.TestLdapGroupsMapping,testLdapConnectionTimeout
32,org.apache.hadoop.hdfs.TestFileChecksum,testStripedFileChecksumWithMissedDataBlocksRan...
33,org.apache.hadoop.hdfs.TestFileChecksum,testStripedFileChecksumWithMissedDataBlocksRan...


In [141]:
df = pd.read_csv('output-tests.csv')
# Give column names
df.columns = ['Tests_run', 'Failures', 'Errors', 'Skipped', 'Test_group']
# Erase all unnecessary characters in each column and transform them into type int64
df['Tests_run'] = df['Tests_run'].str.split(':', n=1).str[1].astype('int64')
df['Failures'] = df['Failures'].str.split(':', n=1).str[1].astype('int64')
df['Errors'] = df['Errors'].str.split(':', n=1).str[1].astype('int64')
df['Skipped'] = df['Skipped'].str.split(':', n=1).str[1].astype('int64')
df['Test_group'] = df['Test_group'].str.split('-', n=1).str[1].str.replace('in','')
# delete lines where no testname is given which corresponds to the total columns
df = df.dropna(subset=['Test_group'])
df

,Tests_run,Failures,Errors,Skipped,Test_group
2,6,0,0,0,org.apache.spark.util.kvstore.InMemoryStoreS...
3,1,0,0,0,org.apache.spark.util.kvstore.ArrayWrappersS...
4,38,0,0,0,org.apache.spark.util.kvstore.LevelDBIterato...
5,38,0,0,0,org.apache.spark.util.kvstore.InMemoryIterat...
6,10,0,0,0,org.apache.spark.util.kvstore.LevelDBTypeInf...
...,...,...,...,...,...
171,1,0,0,0,org.apache.spark.streamg.kafka010.JavaLocati...
172,1,0,0,0,org.apache.spark.streamg.kafka010.JavaDirect...
173,1,0,0,0,org.apache.spark.streamg.kafka010.JavaKafkaR...
177,1,0,0,0,org.apache.spark.streamg.flume.JavaFlumeStre...


In [135]:
df2 = pd.read_csv('java-test-failures.txt', header = None, delimiter='/t', engine='python')
# Name the single column Tests
df2.columns = ['Tests']
# Split the columns into two coulumns named Failed_test and Test_group
df2[['Failed_test', 'Test_group']] = df2['Tests'].str.split('(', n=1, expand= True)
# Remove the column Tests
df2.drop(columns=['Tests'], inplace= True)
# Split the columns Test_group into two after the )
df2[['Test_group', 'unnecsessary']] = df2['Test_group'].str.split(')', n=1, expand= True)
# Remove evrything after the parnetheses
df2.drop(columns=['unnecsessary'], inplace= True)
# Reorder columns
df2= df2[['Test_group', 'Failed_test']]
df2.drop_duplicates()


In [137]:
df2 = pd.read_csv('aborted-tests.txt', header=None, sep='.txt:', engine='python')
# Give column names
df2.columns = ['Test_group','Aborted_tests']
# Iliminate the path to SparkTestSuite file and keep only the module name 
df2['Test_group'] = df2['Test_group'].str.replace("/target/surefire-reports/SparkTestSuite","")
# Merge df1 and df2
df2
#df1 = pd.concat([df1, df2] , ignore_index=True)

,Test_group,Aborted_tests
0,tools,*** RUN ABORTED ***


In [6]:
import json
import pandas as pd

with open(f'results-1.json', 'r') as json_file:
    json_data = json.load(json_file)

# Convert the nested dictionary into a DataFrame
# Create a list called record
    records = []
    # Loop through each element of the json file
    for test_group, values in json_data.items():
        # For each element in the failed_test list
        for failed_test in values['Failed_tests']:
            # Get the class Test_group and define it as a variable test_group
            record = {'Test_group': test_group}
            # Get the attribute values
            record.update(values['attributes'])
            # Get the list of failed tests
            record['Failed_test'] = failed_test
            # Append all these elements to the list records
            records.append(record)

# Define the dataframe
df = pd.DataFrame(records)
df.drop(columns= ['Succeeded', 'Failed', 'Skipped', 'Pending'], inplace=True)

# Read the JSON data as a dictionary
with open('results-10.json', 'r') as json_file:
    json_data = json.load(json_file)

# Convert the nested dictionary into a DataFrame
# Create a list called record
records = []
# Loop through each element of the json file
for test_group, values in json_data.items():
    # For each element in the failed_test list
    for failed_test in values['Failed_tests']:
        # Get the class Test_group and define it as a variable test_group
        record = {'Test_group': test_group}
        # Get the attribute values
        record.update(values['attributes'])
        # Get the list of failed tests
        record['Failed_test'] = failed_test
        # Append all these elements to the list records
        records.append(record)

# Define the dataframe
df_external = pd.DataFrame(records)
# Drop columns that we do not compare
df_external.drop(columns= ['Succeeded', 'Failed', 'Skipped', 'Pending'], inplace=True)


print(df)
print(df_external)
#df = df.merge(df_external, how='left', indicator=True)


                                            Test_group        Aborted_tests  \
0      org.apache.spark.util.kvstore.InMemoryStoreS...                  NaN   
1      org.apache.spark.util.kvstore.ArrayWrappersS...                  NaN   
2      org.apache.spark.util.kvstore.LevelDBIterato...                  NaN   
3      org.apache.spark.util.kvstore.InMemoryIterat...                  NaN   
4      org.apache.spark.util.kvstore.LevelDBTypeInf...                  NaN   
..                                                 ...                  ...   
719                                           sql/hive                  NaN   
720                                           sql/hive                  NaN   
721                                           sql/hive                  NaN   
722                              sql/hive-thriftserver                  NaN   
723                                              tools  *** RUN ABORTED ***   

                                           Failed_t

numpy.float64